In [3]:
# ================================
# HYBRID AWS ARCHITECTURAL RECOMMENDER
# ML (TF-IDF) + DistilBERT
# ================================

import pandas as pd
import numpy as np
import re
import joblib
from typing import Tuple

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Torch / Transformers
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    get_linear_schedule_with_warmup,
)
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load final 11,000-row dataset (10k clean + 1k noisy)
df = pd.read_csv("aws_architectural_recommender_dataset_11000.csv")

print("Original shape:", df.shape)
print(df.columns)

# Keep only the columns we need for classification
df = df[["requirement_text", "primary_service"]].copy()

# Drop rows with missing or empty values
df = df.dropna(subset=["requirement_text", "primary_service"])
df["requirement_text"] = df["requirement_text"].astype(str).str.strip()
df["primary_service"] = df["primary_service"].astype(str).str.strip()

df = df[df["requirement_text"].astype(bool)]
df = df[df["primary_service"].astype(bool)]

df.reset_index(drop=True, inplace=True)
print("After cleaning:", df.shape)
df.head()

def preprocess(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_text"] = df["requirement_text"].apply(preprocess)
df = df[df["clean_text"].astype(bool)].reset_index(drop=True)

print("After preprocessing:", df.shape)
df.head()

# Train/validation/test split
X = df["clean_text"]
y = df["primary_service"]

# Encode labels as integers for DistilBERT
label_encoder = LabelEncoder()
y_ids = label_encoder.fit_transform(y)
num_labels = len(label_encoder.classes_)
print("Number of classes:", num_labels)
print("Classes:", list(label_encoder.classes_)[:10], "...")

# First split: train + temp
X_train, X_temp, y_train_ids, y_temp_ids = train_test_split(
    X,
    y_ids,
    test_size=0.3,           # 70% train, 30% temp
    random_state=42,
    stratify=y_ids
)

# Second split: validation + test from temp
X_val, X_test, y_val_ids, y_test_ids = train_test_split(
    X_temp,
    y_temp_ids,
    test_size=0.5,           # 15% val, 15% test
    random_state=42,
    stratify=y_temp_ids
)

print("Train size:", len(X_train))
print("Val size:", len(X_val))
print("Test size:", len(X_test))

# For ML, we also need string labels (not just ids)
y_train_str = label_encoder.inverse_transform(y_train_ids)
y_val_str   = label_encoder.inverse_transform(y_val_ids)
y_test_str  = label_encoder.inverse_transform(y_test_ids)


vectorizer = TfidfVectorizer(
    max_features=7000,
    ngram_range=(1, 2),
    sublinear_tf=True,
)

X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec   = vectorizer.transform(X_val)
X_test_vec  = vectorizer.transform(X_test)

print("TF-IDF shapes:", X_train_vec.shape, X_val_vec.shape, X_test_vec.shape)

ml_models = {
    "LinearSVC": LinearSVC(),
    "LogisticRegression": LogisticRegression(max_iter=5000, n_jobs=-1),
    "MultinomialNB": MultinomialNB(),
    "RandomForest": RandomForestClassifier(n_estimators=400, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
}

ml_results = {}

for name, model in ml_models.items():
    print(f"\nTraining ML model: {name}")
    model.fit(X_train_vec, y_train_str)
    y_pred = model.predict(X_val_vec)

    acc = accuracy_score(y_val_str, y_pred)
    f1  = f1_score(y_val_str, y_pred, average="weighted")

    ml_results[name] = (acc, f1)
    print(f"{name} Val Accuracy: {acc:.4f}")
    print(f"{name} Val F1-score: {f1:.4f}")
    print("-" * 50)

ml_results_df = pd.DataFrame(ml_results, index=["Accuracy", "F1"]).T
ml_results_df.sort_values("F1", ascending=False)

best_ml_name = ml_results_df["F1"].idxmax()
best_ml_model = ml_models[best_ml_name]

# Retrain best ML model on train + val for stronger final model
X_train_val_vec = vectorizer.fit_transform(pd.concat([X_train, X_val]))
y_train_val_str = np.concatenate([y_train_str, y_val_str])

best_ml_model.fit(X_train_val_vec, y_train_val_str)

print(f"Best ML model: {best_ml_name}")

# Save ML artifacts
joblib.dump(best_ml_model, "best_aws_classifier.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("Saved: best_aws_classifier.pkl, tfidf_vectorizer.pkl, label_encoder.pkl")

# Re-vectorize test using refit vectorizer (train+val)
X_test_vec_final = vectorizer.transform(X_test)

y_test_pred_ml = best_ml_model.predict(X_test_vec_final)

ml_test_acc = accuracy_score(y_test_str, y_test_pred_ml)
ml_test_f1  = f1_score(y_test_str, y_test_pred_ml, average="weighted")

print("=== Best ML Model Test Performance ===")
print("Accuracy:", ml_test_acc)
print("Weighted F1:", ml_test_f1)
print("\nClassification Report:\n")
print(classification_report(y_test_str, y_test_pred_ml))

# We will use the original "requirement_text" (not over-cleaned) for DistilBERT
df_bert = df[["requirement_text", "primary_service"]].copy()
df_bert["label_id"] = label_encoder.transform(df_bert["primary_service"])

# Use the same splits as earlier by index alignment
# For simplicity, reconstruct using indices
train_mask = df["clean_text"].isin(X_train)
val_mask   = df["clean_text"].isin(X_val)
test_mask  = df["clean_text"].isin(X_test)

bert_train = df_bert[train_mask].reset_index(drop=True)
bert_val   = df_bert[val_mask].reset_index(drop=True)
bert_test  = df_bert[test_mask].reset_index(drop=True)

print("BERT Train size:", bert_train.shape)
print("BERT Val size:", bert_val.shape)
print("BERT Test size:", bert_test.shape)
bert_train.head()

class AWSDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = list(texts)
        self.labels = list(labels)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text  = str(self.texts[idx])
        label = int(self.labels[idx])

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item["labels"] = torch.tensor(label, dtype=torch.long)
        return item

model_name = "distilbert-base-uncased"

tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

train_dataset = AWSDataset(
    texts=bert_train["requirement_text"],
    labels=bert_train["label_id"],
    tokenizer=tokenizer,
    max_length=128
)
val_dataset = AWSDataset(
    texts=bert_val["requirement_text"],
    labels=bert_val["label_id"],
    tokenizer=tokenizer,
    max_length=128
)
test_dataset = AWSDataset(
    texts=bert_test["requirement_text"],
    labels=bert_test["label_id"],
    tokenizer=tokenizer,
    max_length=128
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label={i: l for i, l in enumerate(label_encoder.classes_)},
    label2id={l: i for i, l in enumerate(label_encoder.classes_)},
)

model.to(device)

EPOCHS = 5
LR = 2e-5

optimizer = AdamW(model.parameters(), lr=LR)

total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

def train_one_epoch(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item() * input_ids.size(0)

        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total   += labels.size(0)

    avg_loss = total_loss / total
    acc = correct / total

    print(f"Epoch {epoch+1} Train Loss: {avg_loss:.4f} | Train Acc: {acc:.4f}")


def eval_one_epoch():
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids      = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels         = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item() * input_ids.size(0)

            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    avg_loss = total_loss / total
    acc = correct / total
    f1  = f1_score(all_labels, all_preds, average="weighted")

    print(f"Val Loss: {avg_loss:.4f} | Val Acc: {acc:.4f} | Val F1: {f1:.4f}")
    return avg_loss, acc, f1

best_val_f1 = 0.0

for epoch in range(EPOCHS):
    print(f"\n===== Epoch {epoch+1}/{EPOCHS} =====")
    train_one_epoch(epoch)
    val_loss, val_acc, val_f1 = eval_one_epoch()

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        print("New best DistilBERT model, saving checkpoint...")
        model.save_pretrained("distilbert_aws_model")
        tokenizer.save_pretrained("distilbert_aws_model")


# Load best saved DistilBERT model
best_bert_model = DistilBertForSequenceClassification.from_pretrained("distilbert_aws_model")
best_bert_model.to(device)
best_bert_model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for batch in test_loader:
        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["labels"].to(device)

        outputs = best_bert_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

bert_test_acc = accuracy_score(all_labels, all_preds)
bert_test_f1  = f1_score(all_labels, all_preds, average="weighted")

print("=== DistilBERT Test Performance ===")
print("Accuracy:", bert_test_acc)
print("Weighted F1:", bert_test_f1)

print("\nClassification Report:\n")
print(classification_report(
    [label_encoder.classes_[i] for i in all_labels],
    [label_encoder.classes_[i] for i in all_preds]
))

# Reload ML artifacts
ml_model = joblib.load("best_aws_classifier.pkl")
tfidf    = joblib.load("tfidf_vectorizer.pkl")
le       = joblib.load("label_encoder.pkl")

# Reload DistilBERT
bert_model = DistilBertForSequenceClassification.from_pretrained("distilbert_aws_model").to(device)
bert_tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert_aws_model")
bert_model.eval()

def ml_predict_with_confidence(text: str) -> Tuple[str, float]:
    clean = preprocess(text)
    if not clean:
        return "", 0.0
    vec = tfidf.transform([clean])

    # Try to approximate confidence
    if hasattr(ml_model, "predict_proba"):
        proba = ml_model.predict_proba(vec)[0]
        idx = np.argmax(proba)
        conf = float(proba[idx])
        label = ml_model.classes_[idx]
    elif hasattr(ml_model, "decision_function"):
        scores = ml_model.decision_function(vec)
        if scores.ndim == 1:
            scores = scores.reshape(1, -1)
        # softmax over scores
        exp_scores = np.exp(scores - scores.max())
        proba = exp_scores / exp_scores.sum(axis=1, keepdims=True)
        idx = np.argmax(proba)
        conf = float(proba[0, idx])
        label = ml_model.classes_[idx]
    else:
        # fallback: no proper confidence
        label = ml_model.predict(vec)[0]
        conf = 0.5

    return label, conf

def distilbert_predict(text: str) -> str:
    encoding = bert_tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt",
    )
    input_ids      = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        pred_id = torch.argmax(logits, dim=1).item()

    label = le.inverse_transform([pred_id])[0]
    return label

def hybrid_recommend_aws_service(text: str, ml_threshold: float = 0.75) -> Tuple[str, str, float]:
    """
    Returns:
        recommended_service, source_model ('ML' or 'DistilBERT'), ml_confidence
    """
    ml_label, ml_conf = ml_predict_with_confidence(text)

    if ml_label and ml_conf >= ml_threshold:
        return ml_label, "ML", ml_conf
    else:
        bert_label = distilbert_predict(text)
        return bert_label, "DistilBERT", ml_conf
        print("\n================ AWS ARCHITECTURAL RECOMMENDER — USER INPUT MODE ================\n")
print("Type your AWS requirement below. Type 'exit' to stop.\n")

while True:
    user_text = input("Your Query: ").strip()

    if user_text.lower() == "exit":
        print("\nExiting AWS Recommender System. Goodbye!")
        break

    if len(user_text) < 3:
        print("⚠️ Please enter a valid requirement.\n")
        continue

    # Hybrid inference
    service, source, conf = hybrid_recommend_aws_service(user_text)

    print("\n-----------------------------------------------")
    print(f"🔍 User Query: {user_text}")
    print(f"✅ Recommended AWS Service: {service}")
    print(f"🤖 Model Used: {source}")
    print(f"📊 ML Confidence: {conf:.3f}")
    print("-----------------------------------------------\n")



Using device: cuda
Original shape: (11000, 7)
Index(['requirement_text', 'primary_service', 'service_category',
       'complexity_level', 'deployment_scenario', 'keywords_extracted',
       'reference_url'],
      dtype='object')
After cleaning: (11000, 2)
After preprocessing: (11000, 3)
Number of classes: 20
Classes: ['AWS Backup', 'AWS Fargate', 'AWS IAM', 'AWS KMS', 'AWS Lambda', 'AWS Secrets Manager', 'AWS WAF', 'Amazon Aurora', 'Amazon CloudFront', 'Amazon DynamoDB'] ...
Train size: 7700
Val size: 1650
Test size: 1650
TF-IDF shapes: (7700, 2134) (1650, 2134) (1650, 2134)

Training ML model: LinearSVC
LinearSVC Val Accuracy: 1.0000
LinearSVC Val F1-score: 1.0000
--------------------------------------------------

Training ML model: LogisticRegression
LogisticRegression Val Accuracy: 1.0000
LogisticRegression Val F1-score: 1.0000
--------------------------------------------------

Training ML model: MultinomialNB
MultinomialNB Val Accuracy: 1.0000
MultinomialNB Val F1-score: 1.0000

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===== Epoch 1/5 =====
Epoch 1 Train Loss: 1.1046 | Train Acc: 0.8010
Val Loss: 0.0363 | Val Acc: 1.0000 | Val F1: 1.0000
New best DistilBERT model, saving checkpoint...

===== Epoch 2/5 =====
Epoch 2 Train Loss: 0.0270 | Train Acc: 1.0000
Val Loss: 0.0092 | Val Acc: 1.0000 | Val F1: 1.0000

===== Epoch 3/5 =====
Epoch 3 Train Loss: 0.0102 | Train Acc: 1.0000
Val Loss: 0.0046 | Val Acc: 1.0000 | Val F1: 1.0000

===== Epoch 4/5 =====
Epoch 4 Train Loss: 0.0061 | Train Acc: 1.0000
Val Loss: 0.0032 | Val Acc: 1.0000 | Val F1: 1.0000

===== Epoch 5/5 =====
Epoch 5 Train Loss: 0.0048 | Train Acc: 1.0000
Val Loss: 0.0028 | Val Acc: 1.0000 | Val F1: 1.0000
=== DistilBERT Test Performance ===
Accuracy: 0.9997331198291967
Weighted F1: 0.9997330428723782

Classification Report:

                        precision    recall  f1-score   support

            AWS Backup       1.00      1.00      1.00       214
           AWS Fargate       1.00      1.00      1.00       173
               AWS IAM     

In [4]:
!pip install gradio --quiet

import gradio as gr

def gradio_hybrid_predict(user_text):
    if not user_text.strip():
        return "❌ Please enter a valid requirement.", "", ""

    service, source, conf = hybrid_recommend_aws_service(user_text)

    return (
        f"Recommended AWS Service: {service}",
        f"Model Used: {source}",
        f"ML Confidence: {conf:.3f}"
    )

# Build Gradio Interface
iface = gr.Interface(
    fn=gradio_hybrid_predict,
    inputs=gr.Textbox(lines=3, placeholder="Describe your AWS requirement here..."),
    outputs=[
        gr.Textbox(label="AWS Recommendation"),
        gr.Textbox(label="Model Used"),
        gr.Textbox(label="Confidence Score"),
    ],
    title="AWS Architectural Recommender (Hybrid ML + DistilBERT)",
    description="Enter any requirement. Hybrid system will predict the best AWS service."
)

# Launch with public link for Colab
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f35578501d4d7d6d9f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
# ================================
# MODERN FRONTEND (GRADIO UI)
# AWS Hybrid Architectural Recommender
# ================================

!pip install gradio==4.16.0 --quiet

import gradio as gr

# --- Frontend prediction wrapper ---
def gradio_hybrid_predict(user_text):
    if not user_text.strip():
        return (
            "❌ Please enter an AWS requirement.",
            "",
            "",
        )

    service, source, conf = hybrid_recommend_aws_service(user_text)

    # Pretty formatting
    service_line = f"🚀 Recommended AWS Service: **{service}**"
    model_line   = f"🤖 Model Selected: **{source}**"
    conf_line    = f"📊 ML Confidence Score: **{conf:.3f}**"

    return (service_line, model_line, conf_line)


# --- Beautiful UI Styling ---
theme = gr.themes.Soft(
    primary_hue="blue",
    secondary_hue="violet",
    text_size="lg",
    radius_size="lg"
)

# --- Build Interface ---
with gr.Blocks(theme=theme, css="""
    #title {text-align: center; font-size: 2.4rem;}
    #subtitle {text-align: center; margin-top: -10px; opacity: 0.75;}
    .gr-input textarea {font-size: 1.2rem;}
    .gr-output textarea {font-size: 1.2rem;}
""") as frontend:

    gr.Markdown("<h1 id='title'>🔍 AWS Architectural Recommender</h1>")
    gr.Markdown("<p id='subtitle'>Powered by Hybrid ML + DistilBERT Intelligence</p>")

    with gr.Row():
        with gr.Column(scale=1):
            user_text = gr.Textbox(
                label="Describe your AWS requirement",
                placeholder="Example: I need a serverless compute service that runs code on events...",
                lines=4
            )
            submit_btn = gr.Button("✨ Recommend AWS Service", variant="primary")
            clear_btn = gr.Button("🗑️ Clear")

        with gr.Column(scale=1):
            result_service = gr.Markdown()
            result_model   = gr.Markdown()
            result_conf    = gr.Markdown()

    submit_btn.click(
        fn=gradio_hybrid_predict,
        inputs=user_text,
        outputs=[result_service, result_model, result_conf]
    )

    clear_btn.click(
        None,
        None,
        [result_service, result_model, result_conf, user_text],
        js="(x,y,z,w) => ['', '', '', '']"
    )

# Launch and create public link
frontend.launch(share=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.19.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 11.0.3 which is incompatible.
yfinance 0.2.66 requires websockets>=13.0, but you have websockets 11.0.3 which is incompatible.
google-genai 1.52.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 11.0.3 which is incompatible.
jax 0.7.2 requires numpy>=2.0, b

/tmp/ipython-input-3413413634.py:38: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=theme, css="""
/tmp/ipython-input-3413413634.py:38: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=theme, css="""
/usr/local/lib/python3.12/dist-packages/gradio/analytics.py:93: UserWarning: IMPORTANT: You are using gradio version 4.16.0, however version 4.44.1 is available, please upgrade. 
--------
  f"IMPORTANT: You are using gradio version {current_pkg_version}, "


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e7a53ce7ab66c2b8ac.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
